In [1]:
#Updated Model

In [2]:
# Updates to the model:
# 1. Adding two more hidden layers 
# 2. Running the model with a different number of features to see if accuracy and loss are affected
# 3. Altering activation functions to see if model accuracy improves 
# 4. Tested out different models such as a random forest model

In [3]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import psycopg2

In [ ]:
# Postgres username, password, and database name (change accordingly)
postgres_address = 'localhost'
postgres_port = '5432' 
postgres_username = 'postgres' 
postgres_password = '********'
postgres_dbname = 'cardio_disease_db'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=postgres_username,
                        password=postgres_password,
                        ipaddress=postgres_address,
                        port=postgres_port,
                        dbname=postgres_dbname))
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
# Upload csv if cannot connect to database
cardio_train_df = pd.read_csv('cardio_training_clean.csv')
cardio_train_df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI
0,0,50,2,168,136.7,110,80,1,1,0,0,1,0,3.4
1,1,55,1,156,187.4,140,90,3,1,0,0,1,1,5.4
2,2,52,1,165,141.1,130,70,3,1,0,0,0,1,3.6
3,3,48,2,169,180.8,150,100,1,1,0,0,1,1,4.5
4,4,48,1,156,123.5,100,60,1,1,0,0,0,0,3.6


In [5]:
# Define feature data
# features: age, gender, height, weight, ap_hi, 
# ap_lo, cholesterol, gluc, smoke, alco, active 
X = cardio_train_df.drop(['id','cardio'],1).values

In [6]:
# Define target data
# cardio (if the person has cardiovascular disease or not) 
y = cardio_train_df['cardio']

In [7]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [8]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

In [9]:
# Fit the StandardScaler
X_scaler.fit(X_train)

StandardScaler()

In [10]:
# Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 6
hidden_nodes_layer5 = 6

nn_model = tf.keras.models.Sequential()

In [12]:
# First hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

In [13]:
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [14]:
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

In [15]:
# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

In [16]:
# Fifth hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

In [17]:
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [18]:
# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                195       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 498
Trainable params: 498
Non-trainable params: 0
__________________________________________________________

In [19]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=300)

Epoch 1/300
1582/1582 [==============================] - 2s 693us/step - loss: 0.5974 - accuracy: 0.6771
Epoch 2/300
1582/1582 [==============================] - 1s 706us/step - loss: 0.5536 - accuracy: 0.7264
Epoch 3/300
1582/1582 [==============================] - 1s 712us/step - loss: 0.5445 - accuracy: 0.7321
Epoch 4/300
1582/1582 [==============================] - 1s 666us/step - loss: 0.5449 - accuracy: 0.7319
Epoch 5/300
1582/1582 [==============================] - 1s 720us/step - loss: 0.5463 - accuracy: 0.7287
Epoch 6/300
1582/1582 [==============================] - 1s 712us/step - loss: 0.5462 - accuracy: 0.7321
Epoch 7/300
1582/1582 [==============================] - 1s 718us/step - loss: 0.5472 - accuracy: 0.7302
Epoch 8/300
1582/1582 [==============================] - 1s 677us/step - loss: 0.5450 - accuracy: 0.7322
Epoch 9/300
1582/1582 [==============================] - 1s 668us/step - loss: 0.5440 - accuracy: 0.7342
Epoch 10/300
1582/1582 [==============================]

1582/1582 [==============================] - 1s 680us/step - loss: 0.5393 - accuracy: 0.7357
Epoch 79/300
1582/1582 [==============================] - 1s 695us/step - loss: 0.5404 - accuracy: 0.7343
Epoch 80/300
1582/1582 [==============================] - 1s 695us/step - loss: 0.5389 - accuracy: 0.7355
Epoch 81/300
1582/1582 [==============================] - 1s 764us/step - loss: 0.5409 - accuracy: 0.7353
Epoch 82/300
1582/1582 [==============================] - 1s 680us/step - loss: 0.5380 - accuracy: 0.7356
Epoch 83/300
1582/1582 [==============================] - 1s 651us/step - loss: 0.5394 - accuracy: 0.7364
Epoch 84/300
1582/1582 [==============================] - 1s 665us/step - loss: 0.5386 - accuracy: 0.7347
Epoch 85/300
1582/1582 [==============================] - 1s 665us/step - loss: 0.5390 - accuracy: 0.7367
Epoch 86/300
1582/1582 [==============================] - 1s 638us/step - loss: 0.5420 - accuracy: 0.7330
Epoch 87/300
1582/1582 [==============================] - 1

1582/1582 [==============================] - 1s 640us/step - loss: 0.5376 - accuracy: 0.7363
Epoch 155/300
1582/1582 [==============================] - 1s 651us/step - loss: 0.5392 - accuracy: 0.7360
Epoch 156/300
1582/1582 [==============================] - 1s 653us/step - loss: 0.5397 - accuracy: 0.7366
Epoch 157/300
1582/1582 [==============================] - 1s 647us/step - loss: 0.5413 - accuracy: 0.7366
Epoch 158/300
1582/1582 [==============================] - 1s 643us/step - loss: 0.5397 - accuracy: 0.7374
Epoch 159/300
1582/1582 [==============================] - 1s 686us/step - loss: 0.5372 - accuracy: 0.7391
Epoch 160/300
1582/1582 [==============================] - 1s 641us/step - loss: 0.5386 - accuracy: 0.7355
Epoch 161/300
1582/1582 [==============================] - 1s 645us/step - loss: 0.5394 - accuracy: 0.7362
Epoch 162/300
1582/1582 [==============================] - 1s 678us/step - loss: 0.5410 - accuracy: 0.7361
Epoch 163/300
1582/1582 [==========================

1582/1582 [==============================] - 1s 654us/step - loss: 0.5376 - accuracy: 0.7378
Epoch 231/300
1582/1582 [==============================] - 1s 644us/step - loss: 0.5355 - accuracy: 0.7403
Epoch 232/300
1582/1582 [==============================] - 1s 648us/step - loss: 0.5369 - accuracy: 0.7383
Epoch 233/300
1582/1582 [==============================] - 1s 645us/step - loss: 0.5368 - accuracy: 0.7395
Epoch 234/300
1582/1582 [==============================] - 1s 645us/step - loss: 0.5359 - accuracy: 0.7390
Epoch 235/300
1582/1582 [==============================] - 1s 655us/step - loss: 0.5413 - accuracy: 0.7379
Epoch 236/300
1582/1582 [==============================] - 1s 651us/step - loss: 0.5335 - accuracy: 0.7406
Epoch 237/300
1582/1582 [==============================] - 1s 643us/step - loss: 0.5410 - accuracy: 0.7353
Epoch 238/300
1582/1582 [==============================] - 1s 636us/step - loss: 0.5319 - accuracy: 0.7413
Epoch 239/300
1582/1582 [==========================

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

528/528 - 0s - loss: 0.5459 - accuracy: 0.7313
Loss: 0.5458629727363586, Accuracy: 0.7313140630722046
